## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
filtered_data = data.loc[
                               (data.GS > 20) |
                               ((data.MP/ data.G) > 10)
]

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
X = filtered_data.loc[
                         :, 
                         ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']
                         ]

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here
ss= StandardScaler()
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [6]:
# answer goes here
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
kmeans = KMeans(n_clusters=3)
kmeans.fit_predict(X_std)

centers_inverse = pd.DataFrame(ss.inverse_transform(kmeans.cluster_centers_), columns = X_std.columns)
centers_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765
1,10.438829,3.982430,2.033406,0.800217,0.383731,1.230369
2,13.954913,8.713873,1.936416,0.779769,1.282081,1.694798


In [7]:
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=3)
gmm.fit_predict(X_std)

gmm_centers_inverse = pd.DataFrame(ss.inverse_transform(gmm.means_), columns=X_std.columns)
gmm_centers_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,15.850806,4.117929,4.766841,1.155442,0.358102,2.230856
1,10.140714,4.568029,1.562644,0.767764,0.449156,1.105237
2,14.500210,8.387983,2.269705,0.816923,1.242266,1.832581


In [8]:
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
kmeans = KMeans(n_clusters=7)
kmeans.fit_predict(X_std)

centers_inverse = pd.DataFrame(ss.inverse_transform(kmeans.cluster_centers_), columns = X_std.columns)
centers_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,7.516230,3.509424,1.336649,0.593717,0.362827,0.895288
1,11.330000,4.957778,2.106667,1.370000,0.507778,1.278889
2,24.603226,8.374194,7.803226,1.603226,0.809677,3.941935
3,14.249123,3.632749,3.218129,0.850292,0.271345,1.712281
4,17.568519,10.762963,2.211111,0.918519,1.885185,2.164815
5,11.945455,7.374026,1.746753,0.659091,0.955844,1.433117
6,18.627869,4.728689,5.812295,1.338525,0.411475,2.643443


In [9]:
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=7)
gmm.fit_predict(X_std)

gmm_centers_inverse = pd.DataFrame(ss.inverse_transform(gmm.means_), columns=X_std.columns)
gmm_centers_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,11.041348,5.629386,2.277014,1.159108,0.613531,1.474562
1,20.728854,8.399230,6.285991,1.341743,1.104938,3.217511
2,14.715161,3.102003,3.959651,0.886172,0.220287,1.885143
3,18.838589,9.336168,2.942602,0.867723,1.494193,2.426178
4,17.072676,4.789001,5.510582,1.455942,0.391268,2.467922
5,12.864364,7.190739,1.485758,0.598289,0.948308,1.415522
6,8.964333,3.994269,1.418420,0.704386,0.417408,0.961637


Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [10]:
# answer goes here
X_std = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=3)
gmm.fit(X_std)
probs = pd.DataFrame(gmm.predict_proba(X_std))
ent = entr(probs)

In [11]:
total_ent = pd.DataFrame(ent.sum(axis=1), columns=['Total Entropy'])
total_ent

,Total Entropy
0,0.693695
1,0.555440
2,0.028199
3,0.144929
4,0.385610
...,...
808,0.006110
809,0.713562
810,0.493063
811,0.597019


In [12]:
full_ent = pd.concat([data, total_ent], axis=1)

In [13]:
# Least amount of certainty/Highest Entropy
full_ent.sort_values(by='Total Entropy', ascending=False).head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Total Entropy
13,2019,Justin Anderson,SF,25,ATL,48,4,9.6,1.3,3.3,0.408,0.5,1.6,0.312,0.8,1.7,0.500,0.484,0.5,0.7,0.743,0.5,1.3,1.8,0.5,0.5,0.3,0.5,1.0,3.7,1.096038
750,2018,Justin Holiday,SG,28,CHI,72,72,31.5,4.0,10.9,0.371,2.2,6.2,0.359,1.8,4.6,0.387,0.474,1.9,2.3,0.823,0.4,3.6,4.0,2.1,1.1,0.4,1.3,1.9,12.2,1.078299
671,2018,Kris Dunn,PG,23,CHI,52,43,29.3,5.5,12.8,0.429,0.8,2.6,0.321,4.7,10.2,0.457,0.462,1.6,2.2,0.737,0.5,3.8,4.3,6.0,2.0,0.5,2.9,3.3,13.4,1.065902
687,2018,Raymond Felton,PG,33,OKC,82,2,16.6,2.7,6.7,0.406,1.0,2.8,0.352,1.7,3.9,0.444,0.479,0.4,0.5,0.818,0.3,1.6,1.9,2.5,0.6,0.2,0.9,1.1,6.9,0.982467
728,2018,Joe Harris,SG,26,BRK,78,14,25.3,3.9,8.0,0.491,1.9,4.6,0.419,2.0,3.4,0.589,0.612,1.1,1.3,0.827,0.7,2.7,3.3,1.6,0.4,0.3,1.2,2.0,10.8,0.972576


In [14]:
# Most amount of certainty/Lowest Entropy
full_ent.sort_values(by='Total Entropy', ascending=True).head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Total Entropy
593,2018,Malcolm Brogdon,PG,25,MIL,48,20,29.9,5.1,10.5,0.485,1.3,3.4,0.385,3.8,7.1,0.532,0.547,1.6,1.8,0.882,0.5,2.8,3.3,3.2,0.9,0.3,1.4,2.7,13.0,3.723546e-111
806,2018,Luke Kornet,PF,22,NYK,20,1,16.3,2.5,6.3,0.392,1.4,4.0,0.354,1.1,2.3,0.457,0.504,0.4,0.6,0.727,0.6,2.7,3.2,1.3,0.3,0.8,0.6,1.4,6.7,1.277723e-31
355,2019,Malik Monk,SG,20,CHO,73,0,17.2,3.1,8.0,0.387,1.5,4.5,0.330,1.6,3.5,0.461,0.480,1.2,1.4,0.882,0.2,1.7,1.9,1.6,0.5,0.3,1.2,1.5,8.9,2.394789e-18
457,2019,Tony Snell,SF,27,MIL,74,12,17.6,2.2,4.9,0.452,1.1,2.8,0.397,1.1,2.1,0.522,0.564,0.5,0.6,0.881,0.4,1.7,2.1,0.9,0.4,0.2,0.3,1.2,6.0,1.133318e-17
181,2019,Rudy Gay,PF,32,SAS,69,51,26.7,5.4,10.8,0.504,1.1,2.7,0.402,4.4,8.1,0.537,0.554,1.7,2.1,0.816,0.9,5.9,6.8,2.6,0.8,0.5,1.7,2.3,13.7,2.184934e-17
